# 36120 AT1 Kaggle NBA prediction

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
# Load datasets
train_data = pd.read_csv('C:\\Users\\13803\\Downloads\\uts-36120-2024-spr\\train.csv')
test_data = pd.read_csv('C:\\Users\\13803\\Downloads\\uts-36120-2024-spr\\test.csv')

# Quick overview of the dataset
print(train_data.head())
print(train_data.info())


            team conf  GP  Min_per   Ortg   usg   eFG  TS_per  ORB_per  \
0  South Alabama   SB  26     29.5   97.3  16.6  42.5   44.43      1.6   
1       Utah St.  WAC  34     60.9  108.3  14.9  52.4   54.48      3.8   
2  South Florida   BE  27     72.0   96.2  21.8  45.7   47.98      2.1   
3     Pepperdine  WCC  30     44.5   97.7  16.0  53.6   53.69      4.1   
4        Pacific   BW  33     56.2   96.5  22.0  52.8   54.31      8.3   

   DRB_per  ...     dgbpm    oreb    dreb    treb     ast     stl     blk  \
0      4.6  ... -1.941150  0.1923  0.6154  0.8077  1.1923  0.3462  0.0385   
1      6.3  ... -0.247934  0.6765  1.2647  1.9412  1.8235  0.4118  0.2353   
2      8.0  ... -0.883163  0.6296  2.3333  2.9630  1.9630  0.4815  0.0000   
3      9.4  ... -0.393459  0.7000  1.4333  2.1333  1.1000  0.5667  0.1333   
4     18.6  ... -0.668318  1.4242  3.3030  4.7273  0.8485  0.4545  0.3333   

       pts                             player_id  drafted  
0   3.8846  b2716b2d-3422-4959-9

C:\Users\13803\AppData\Local\Temp\ipykernel_10724\195921245.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('C:\\Users\\13803\\Downloads\\uts-36120-2024-spr\\train.csv')


In [4]:
# Check for missing values
print(train_data.isnull().sum())

train_data.fillna(train_data.mean(), inplace=True)
test_data.fillna(test_data.mean(), inplace=True)

team          0
conf          0
GP            0
Min_per       0
Ortg          0
             ..
stl          36
blk          36
pts          36
player_id     0
drafted       0
Length: 64, dtype: int64


C:\Users\13803\AppData\Local\Temp\ipykernel_10724\2399176690.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_data.fillna(train_data.mean(), inplace=True)
C:\Users\13803\AppData\Local\Temp\ipykernel_10724\2399176690.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_data.fillna(test_data.mean(), inplace=True)


In [6]:
# Selected numerical variables based on the heatmap
selected_features = ['Ortg', 'eFG', 'FTM', 'FT_per', 'TP_per', 'stl_per', 'pfr', 
                     'Rec_Rank', 'dunksmade', 'bpm']

X = train_data[selected_features]
y = train_data['drafted']

X_test = test_data[selected_features]

In [7]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [9]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predictions on the validation set
y_val_pred = rf_model.predict(X_val)

In [10]:
# Evaluate the model's performance
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.9906692406692407
Precision: 0.7027027027027027
Recall: 0.2549019607843137
F1 Score: 0.3741007194244604


In [11]:
y_test_prob = rf_model.predict_proba(X_test_scaled)[:, 1] 

y_test_prob_rounded = np.round(y_test_prob, 1)

submission = pd.DataFrame({'player_id': test_data['player_id'], 'draft_probability': y_test_prob_rounded})
submission.to_csv('C:\\Users\\13803\\Downloads\\uts-36120-2024-spr\\sample_submission.csv', index=False)